# Importing all necessary libraries and our data

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [4]:
df = pd.read_csv('/content/gdrive/MyDrive/TheSocialDilemma/TheSocialDilemma.csv')

## Spliting our data into 2 parts   80/20

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['Sentiment'], test_size=0.20)

## Removing neutral comments

In [6]:
X_train = X_train[(df.Sentiment == 'Positive') | (df.Sentiment == 'Negative')]
X_test = X_test[(df.Sentiment == 'Positive') | (df.Sentiment == 'Negative')]
y_train = y_train[(df.Sentiment == 'Positive') | (df.Sentiment == 'Negative')]
y_test = y_test[(df.Sentiment == 'Positive') | (df.Sentiment == 'Negative')]


## Cleaninig text

In [7]:
stop_words = stopwords.words("english")

In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub("@\S+", " ", text)
    text = re.sub("https*\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\d", " ", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\s{2,}',' ', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

In [9]:
X_train = X_train.apply(lambda x:clean_text(x))
X_test = X_test.apply(lambda x:clean_text(x))

## Converting "Positive" and "Negative" into 1.0 and 0.0 for further calculations

In [10]:
def MakingLabel(text):
  if text == "Positive":
    return 1.0
  if text == "Negative":
    return 0.0

In [11]:
y_train = y_train.apply(lambda x:MakingLabel(x))
y_test = y_test.apply(lambda x:MakingLabel(x))

## Making a dictionary of the whole text to create sequences

In [12]:
text = ''
for i in X_train:
    text += i + ' '
for i in X_test:
    text += i + ' '

In [13]:
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens

In [14]:
def lemmatizer(text):
    lemm_text = [WordNetLemmatizer().lemmatize(word) for word in text]
    return lemm_text

In [15]:
dict_full = list(set(lemmatizer(tokenization(text))))

## Making sequences

In [16]:
def sequences(text):
    sequence = [dict_full.index(i) for i in lemmatizer(tokenization(text))]
    return sequence

In [17]:
X_train = X_train.apply(lambda x:sequences(x))
X_test = X_test.apply(lambda x:sequences(x))

## Vectorizing sequences

In [18]:
def vectorize_sequences(sequences, dimension=11000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

In [19]:
X_train = vectorize_sequences(X_train)
X_test = vectorize_sequences(X_test)

In [20]:
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")

# Making our model

In [59]:
maxlen = 120

X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [22]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [23]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [62]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
vocab_size = 11000


inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(30, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [63]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    X_train, y_train, batch_size=32, epochs=5, callbacks=[callback], validation_data=(X_test, y_test)
)

Epoch 1/5
328/328 [==============================] - 36s 102ms/step - loss: 0.6038 - accuracy: 0.7204 - val_loss: 0.5833 - val_accuracy: 0.7301
Epoch 2/5
328/328 [==============================] - 31s 93ms/step - loss: 0.5954 - accuracy: 0.7261 - val_loss: 0.5832 - val_accuracy: 0.7301
Epoch 3/5
328/328 [==============================] - 33s 100ms/step - loss: 0.5921 - accuracy: 0.7263 - val_loss: 0.5836 - val_accuracy: 0.7301
Epoch 4/5
328/328 [==============================] - 32s 99ms/step - loss: 0.5908 - accuracy: 0.7262 - val_loss: 0.5835 - val_accuracy: 0.7301
Epoch 5/5
328/328 [==============================] - 31s 94ms/step - loss: 0.5912 - accuracy: 0.7263 - val_loss: 0.5839 - val_accuracy: 0.7301


In [64]:
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.5838561058044434
Test accuracy: 0.7300613522529602
